# Group information

## List of members and Assignments

| Student ID | Name | Tasks |
| ----------|-------|--------------|
| 21120402 | Trương Hoàng Nhân | ... |
| 21120458 | Nguyễn Huy Hoàng | ... |


# Set up system

And running Mini-CNN for runnability testing.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Check CUDA compiler version

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
%cd /content/drive/MyDrive/Parallel_Programming/Final_project/

/content/drive/MyDrive/Parallel_Programming/Final_project


In [3]:
!git clone https://github.com/Hoangnhan1708/ANN-Final-Project.git

Cloning into 'ANN-Final-Project'...
remote: Enumerating objects: 2018, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 2018 (delta 46), reused 55 (delta 25), pack-reused 1937 (from 1)
Receiving objects: 100% (2018/2018), 50.56 MiB | 14.75 MiB/s, done.
Resolving deltas: 100% (382/382), done.
Updating files: 100% (1779/1779), done.


Set the location to the `CNN_Parallelization` directory as example below

In [4]:
#projectDirRootName = "path/to/CNN_Parallelization/" # Change to correct localtion of CNN_Parallelization directory
projectDirRootName = "/content/drive/MyDrive/Parallel_Programming/Final_project/ANN-Final-Project/"

## Compile executable file

Set up for compiling demo.exe. If the demo.exe has been put in the correct location: skip belows cell until you have met cell "RUN THIS". But you should re-complie and re-linking objects code again to receive the executable for sure.

In [5]:
%cd {projectDirRootName}Source/ann
%ls

/content/drive/MyDrive/Parallel_Programming/Final_project/ANN-Final-Project/Source/ann
CMakeLists.txt  data/                  LICENSE    report.ipynb  test/             testImplement.h
config.h        demo_Fashion_MNIST.cc  readme.md  src/          testImplement.cc  third_party/


In [ ]:
%cd ..

/content/drive/MyDrive/Parallel_Programming/Final_project/ANN-Final-Project/Source/mini-dnn-cpp-master


In [6]:
%rm -r build
%mkdir build
%cd build
%ls

rm: cannot remove 'build': No such file or directory
/content/drive/MyDrive/Parallel_Programming/Final_project/ANN-Final-Project/Source/ann/build


In [7]:
!cmake ..

CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The CXX compiler identification is GNU 11.4.0
-- The CUDA compiler identification is NVIDIA 12.2.140 with host compiler GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Configuring done (5.8s)
-- Generating done (0.4s)
-- Bui

In [18]:
!make

[ -7%] Building CXX object src/CMakeFiles/MiniDNNLib.dir/layer/fully_connected.cc.o
[  0%] Linking CXX static library libMiniDNNLib.a
[ 61%] Built target MiniDNNLib
[ 69%] Linking CXX executable demo
[ 84%] Built target demo


## RUN THIS

Run the `./demo.exe` for activating LeNet-5 running on the test set. The `demo.exe` can run with multiple version of Convolutional layer (currently there are 3 versions which are indexed from 0 to 3) on the same test set, by passing follwing arguments:

- First argument: the index of the first version you want to execute.
- Second argument: the index of final version that `demo.exe` will implement.
- Third argument: if `0`, the `demo.exe` only executes the first version. If `1`, the `demo.exe` executes all versions from first one to the final one.

Following is the list of implemented versions and theirs index (The first and second arguments can gain below number):

- `0` - The sequential version: The forward of Convolutional layer is sequential in the input rolling stage.
- `1` - The first parallel version: The forward of Convolutional layer is parallelized in the input rolling stage.
- `2` - The second parallel version: The forward of Convolutional layer is parallelizedd in the matrix multiplication between input features and layer's weights.
- Any other number - The original implementation of Convolutional layer which is provided as is by authors of `mini-dnn-cpp` project.


### Version 0: Library version

In [ ]:
%cd {projectDirRootName}/Source/ann/build
%ls
!chmod 755 demo
!./demo 0 0 1
# !./demo -1 2 1

/content/drive/MyDrive/Parallel_Programming/Final_project/ANN-Final-Project/Source/mini-dnn-cpp-master/build
CMakeCache.txt  CMakeFiles/  cmake_install.cmake  demo*  Makefile  src/
../data/fashion-mnist/
mnist train number: 60000
mnist test number: 10000
---------------------------Current Version 0----------------------------------
---------Epochs 0----------
0-th batch, loss: 2.33651
50-th batch, loss: 1.67607
100-th batch, loss: 1.02396
150-th batch, loss: 0.819132
200-th batch, loss: 0.687898
250-th batch, loss: 0.712934
300-th batch, loss: 0.721538
350-th batch, loss: 0.661831
400-th batch, loss: 0.649526
450-th batch, loss: 0.628897
Accuracy: 0.7888

---------Epochs 1----------
0-th batch, loss: 0.713357
50-th batch, loss: 0.635713
100-th batch, loss: 0.546469
150-th batch, loss: 0.571192
200-th batch, loss: 0.524243
250-th batch, loss: 0.587108
300-th batch, loss: 0.528695
350-th batch, loss: 0.458974
400-th batch, loss: 0.579662
450-th batch, loss: 0.551308
Accuracy: 0.814

----

~13phút.

### Version 1: Sequential version

In [ ]:
%cd {projectDirRootName}/Source/ann/build
%ls
!chmod 755 demo
!./demo 1 1 1
# !./demo -1 2 1

/content/drive/MyDrive/Parallel_Programming/Final_project/ANN-Final-Project/Source/mini-dnn-cpp-master/build
CMakeCache.txt  CMakeFiles/  cmake_install.cmake  demo*  Makefile  src/
../data/fashion-mnist/
mnist train number: 60000
mnist test number: 10000
---------------------------Current Version 1----------------------------------
---------Epochs 0----------
0-th batch, loss: 2.33651
50-th batch, loss: 1.67607
100-th batch, loss: 1.02393
150-th batch, loss: 0.819054
200-th batch, loss: 0.687945
250-th batch, loss: 0.713016
300-th batch, loss: 0.721229
350-th batch, loss: 0.661694
400-th batch, loss: 0.649563
450-th batch, loss: 0.628735
Weights changed in epoch 0
Accuracy: 0.7886

---------Epochs 1----------
0-th batch, loss: 0.713293
50-th batch, loss: 0.635797
100-th batch, loss: 0.546579
150-th batch, loss: 0.571036
200-th batch, loss: 0.524141
250-th batch, loss: 0.587347
300-th batch, loss: 0.528953
350-th batch, loss: 0.459042
400-th batch, loss: 0.579811
450-th batch, loss: 0.5

~130,915.83 phút

### Version 2: Parallel version (not optimized)

In [19]:
%cd {projectDirRootName}/Source/ann/build
%ls
!chmod 755 demo
!./demo 2 2 1

/content/drive/MyDrive/Parallel_Programming/Final_project/ANN-Final-Project/Source/ann/build
CMakeCache.txt  CMakeFiles/  cmake_install.cmake  demo*  Makefile  src/
../data/fashion-mnist/
mnist train number: 60000
mnist test number: 10000
---------------------------Current Version 2----------------------------------
---------Epochs 0----------
0-th batch, loss: 2.33651
50-th batch, loss: 2.13046
100-th batch, loss: 1.60348
150-th batch, loss: 1.25736
200-th batch, loss: 0.970255
250-th batch, loss: 0.901808
300-th batch, loss: 0.901171
350-th batch, loss: 0.750359
400-th batch, loss: 0.78029
450-th batch, loss: 0.733773
Accuracy: 0.7185

---------Epochs 1----------
0-th batch, loss: 0.83113
50-th batch, loss: 0.743485
100-th batch, loss: 0.671388
150-th batch, loss: 0.681734
200-th batch, loss: 0.630611
250-th batch, loss: 0.688706
300-th batch, loss: 0.672542
350-th batch, loss: 0.540811
400-th batch, loss: 0.653579
450-th batch, loss: 0.651906
Accuracy: 0.7833

---------Epochs 2-----

~17 phút

### Version 3: Parallel version (optimized)

In [20]:
%cd {projectDirRootName}/Source/ann/build
%ls
!chmod 755 demo
!./demo 3 3 1

/content/drive/MyDrive/Parallel_Programming/Final_project/ANN-Final-Project/Source/ann/build
CMakeCache.txt  CMakeFiles/  cmake_install.cmake  demo*  Makefile  src/
../data/fashion-mnist/
mnist train number: 60000
mnist test number: 10000
---------------------------Current Version 3----------------------------------
---------Epochs 0----------
0-th batch, loss: 2.33651
50-th batch, loss: 2.13046
100-th batch, loss: 1.60348
150-th batch, loss: 1.25736
200-th batch, loss: 0.970255
250-th batch, loss: 0.901808
300-th batch, loss: 0.901171
350-th batch, loss: 0.750359
400-th batch, loss: 0.78029
450-th batch, loss: 0.733773
Accuracy: 0.7185

---------Epochs 1----------
0-th batch, loss: 0.83113
50-th batch, loss: 0.743485
100-th batch, loss: 0.671388
150-th batch, loss: 0.681734
200-th batch, loss: 0.630611
250-th batch, loss: 0.688706
300-th batch, loss: 0.672542
350-th batch, loss: 0.540811
400-th batch, loss: 0.653579
450-th batch, loss: 0.651906
Accuracy: 0.7833

---------Epochs 2-----

~16 phút

# Problem definition

# Convolutional Layer - LeNet-5 Model

As described in the previous seminar and slides, the Convolutional layer (Conv. layer for short) is the most basic and important architecture used in the Computer Vision fields. It can reduce the size of input images, as well as capture the lower to higher information of the image and transfer those knowlege to top layers of the model for downstream tasks.

The Conv. layer uses the kernel matrix and convolution operation to achieve the task. However, the time complexity of that computation is really high, especially with large number of images and high quality images. As soon as GPU was utilized for parallelize the computing, people has found a way to speed up the Conv. layer implementation. This project try to use a small subset of CUDA API to test the parallelization capability of GPU in Conv. layer computing.

We built LeNet-5 models with the configuration as suggested in project description. The below figure visualizes the details of LeNet-5.

![](https://drive.google.com/uc?export=view&id=1OQGrV0-1fK5N5naB-JfvXeb0X_WzeDG1)

## Basic idea

We use the given starter project [2] as the staring point for parallelizing the forward phase of Conv. layer. Here, the author uses the matrix multiplication between the weight matrix of Conv. layer and unrolled version of batch input images for computing output of Conv. layer, as shown in below image ([1]).

![](https://drive.google.com/uc?export=view&id=1XMlgD_H-7o6lfOp7WG_5DIXvrt7U-QWk)

## Input/Output and Terms

### Input layout

A mini-batch of multiple input images (input features) $X$ as the tensor has shape $(N, C, H, W)$ where:

* $N$: number of samples in a mini-batch
* $C$: number of input feature maps.
* $H$: height of each input feature map (or simply just the height of each input image in pixels).
* $W$: width of each input feature map (or simply just the weight of each input image in pixels).

### Output layout

The output features after applying CNN to $X$. It is an array $Y$ as the tensor has shape $(N, M, H_\text{out}, W_\text{out})$ where:

* $N$: number of samples in a mini-batch
* $M$: number of output feature maps of a CNN layer.
* $H_\text{out}$: height of each output feature map (it is often that $H_\text{out} = H - K + 1$).
* $W_\text{out}$: width of each output feature map (it is often that $W_\text{out} = W - K + 1$).

### Filter-bank layout

The matrix $W$ contains all filer maxtrix that is used for one CNN layer (or simply just the weigth matrix of a CNN layer).  It is a tensor has shape $(M, C, K, K)$ where:

* $K$: the size of a filter matrix (or kernel matrix).

With an input image has $C$ input feature maps (or $C$ color chanels) and the CNN layer produces $M$ output feature maps from that input feature maps, we need $C \times M$ kernel matrix with size $K$.

### The Unrolled-X

We unroll the matrix X. As a result, we can get all elements that are required for computing all output feature maps from a single input image, with just a single matrix multiplication step between $X$ and $W$. For now, just know that $X_\text{unroll}$ is retrieved from $X$, and it has shape $(C, K, K, H_\text{out}, W_\text{out})$, where:

* $(C, K, K)$: the "height" of $X_\text{unroll}$. It is the number of elements in $X$ that we need to compute an output feature map element (that is $C \times K \times K$),
* $(H_\text{out}, W_\text{out})$: the "width" of $X_\text{unroll}$. It is the number of elements in an output feature map, that is $H_\text{out} \times W_\text{out}$.

## Contribution (Results)



### <a name="tableResult"></a>Table of Results of Implemented Version of LeNet-5 (Conv. Layer)

| ID  | Program specification | Time execution - Forward only<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) (ms) | Test Accuracy | Time of running Test set<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2) (ms) |
| --- | --------------------- | ---------------------------------- | ----------------------- | - |
| A0 | LeNet-5 - Original - MNIST || 0.9737 ||
| **A1** | **LeNet-5 - Original - Fashion-MNSIT** | **$ \approx 1040.63$** | **0.8297** | **78152.4** |
| A2 | LeNet-5 - Original - Transpose Matrix Multiplication | $ \approx 985.677$ | 0.8194 ||
| **B0** | **LeNet-5 - Sequential (Im2Col) - ~~Transpose Matrix Multiplication~~** | **$ \approx 1115.75 \rightarrow 916.248$** | **0.8297** | **73255.4** |
| C0 | LeNet-5 - Parallel Version 1 (Im2Col) - Get unrolled image matrix avoid naive copy | $ \approx 696.307 $ | 0.8297 ||
| **C1** | **LeNet-5 - Parallel Version 1 (Im2Col) - Same as (C0), but get unrolled image results directly from dynamic memory to Matrix object** | **$ \approx 539.801 $** | **0.8297** | **41349.5** |
| **D0** | **LeNet-5 - Parallel Version 2 (MatMul) - Save matrix while multiplying as column-major order** | | **0.8297** | **31260.9** |

<a name="cite_note-1"></a>1. [^](#cite_ref-1) : Avarage of all forward passes in first epoch.

<a name="cite_note-2"></a>2. [^](#cite_ref-2): The Test set contains 10000 samples. The results is the best one after re-run the executable files 10 times.

# Version 0: Naive implementation (Sequential Version)

## Experiment, verification, timing strategy

![](https://drive.google.com/uc?export=view&id=1VrSjbh-X32nRMI0uwcH_hC-ejDFrEPgO)

For all following experiments, we need to have these objects:

1. **Weight matrix of LeNet-5**
  
  To save time for experimenting, we do not need to train the model again and again. We only need to pass the test set to the new version and get the test accuracy. To avoid repeating training and have proper ground truth weigth matrix, we train original version of the LeNet-5 on the full training set with standard configuration, and then save the model weights to file (As drawn in Pre-Train Phase in above figure).

  For any new version of Convolutional layer, we only need to load that saved model weights from file to the model, then run on test set and record observed results (As drawn in Testing Phase in above figure).

2. **Test cases**

  In many cases, the accuracy of the test set is not enough for concluding if the new implementation of Convolutional layer is sufficient, or even worse it may lead to misleading results due to random process of the models. To verify the correctness of the new Convolutional layer, we have built 20 test cases for independent testing of forward step of Conv. layer.

  The 20 test cases is built randomly. 10 of them has shape $(1, 28, 28)$ (the size of batch is not considered while doing test cases) which are the same shape of one input image of the first Conv. layer. 10 remaining test cases has shape $(6, 24, 24)$ which are the same shape of one input features map of the second Conv. layer. All of elements of each test case matrix are in range $[0, 255]$. The expected output of test cases is produced by passing them through the forward function of the Conv. layer of the original LeNet-5 version.

  For each new version Conv. layer, we pass all 20 test cases through the forward step and then, compare the produced results with the expected output test cases (as drawn in purple ovals in the below figure). The pass/fail status of each test case is reported while the program is running.

  All of test cases and theirs expected output is stored in `CNN_Parallelization/Source/mini-dnn-cpp-master/test` directory.

The time measurement of each version is the time to forward all samples in the test cases. In other words, we measure how long the model can process and return the result of the given Fashion-MNIST test set.

![](https://drive.google.com/uc?export=view&id=1XQFtDvNYVNP2CFcGI5lSTQAnpi1iMRIb)

## Implementation and Specification

This version is implemented in the `conv.cc` file. We create `forwardVerion_0()` to activate this forward version, and `unroll()` as the simplified version of original unroll input features maps `im2col()`.

The below is the pseudo-code of this implementation:

```[C++]
foreach input-feature c of image
  foreach position (p, q) of each area
    foreach possible-position-of-applied-kernel-area-KxK area of c
      insert element (p, q) to X-unrolled
```

Simply speaking, we process each input feature $c$ ($c \in [0, C)$) of an image. For each $c$, we want to collect elemen at position ($p, q$) ($p, q \in [0, K)$) of the current area in input feature which is being applied kernel matrix. The ($p, q$) elements is inserted in the row $p \times K + q$ of the X-unrolled matrix. We do the same for each next area when the kernel matrix is slided, which the next elements will be inserted in following columns of X-unrolled matrix.

When we have done for an input feature $c$, we just go to the next row of X-unrolled matrix and repeat the second and third for loop as before.

## Result

Test accuracy and Testing time can be refered to [this table, version A1](#tableResult).

Additionally, this version passed all the designated test cases.

# Version 1: Basic Parallelize

## Basic idea

In the version 0, we can see that each time an $x$-th element belongs to an applied kernel matrix area $i$ has been inserted into the X-unrolled, that area $i$ has to wait until all other $x$-th elements of others area $j$ ($j \geq i$) have also been inserted. We can seperate the insertion of each individual area by letting each thread processes an applied kernel matrix area from input features.

## Implementation and Specification

This version is implemented in the `conv.cc` and `cuda_utilities.cu` file. We create `forwardVersion_1()` to activate this forward version, and `unrollKernel_1()` as the parallelized kernel version of original unroll input features maps `im2col()`. `forwardVersion_1()` activate `unrollKernel_1()` by calling `unrollGPUWrapper()` in `cuda_utilities.cu`.

This version parallelize the third for loop of above sequential version. Now, each thread only need to executes the second for loop, which means each thread inserts the input feature elements belong to the same area applied kernel matrix into the X-unrolled. Because there are $C$ input feature of each image, and there are $H_\text{out} \times W_\text{out}$ possible area which can be applied kernel matrix, we need a grid contains $C \times H_\text{out} \times W_\text{out}$ threads.

## Result

Test accuracy and Testing time can be refered to [this table, version C1](#tableResult).

Additionally, this version passed all the designated test cases.

# Version 2: Matrix Multiplication Parallelization

## Basic idea

By seperating the insertion of each applied kernel matrix area in input features, we can speed up the execution a lot. However, there is still a major bottleneck in the forward step of Conv. layer after unrolling the input matrix: the matrix multiplication between the X-unrolled and weight matrix of the Conv. layer.

After unrolling phase, it is common that we often need to multiply two big size matrix with each other. That is an opportunity to speed up the program by parallelizing the matrix multiplication.

## Implementation and Specification

This version is implemented in the `conv.cc` and `cuda_utilities.cu` file. We create `forwardVersion_2()` to activate this forward version, and `matrixMultiplicationKernel_1()` as the parallelized kernel version of matrix multiplication. `forwardVersion_2()` activate `matrixMultiplicationKernel_1()` by calling `matrixMultiplicationGPUWrapper()` in `cuda_utilities.cu`.

The basic idea of parallelized matrix multiplicatin we have used is shared tiled memory matrix multiplication (As implmeneted in Lab 2).

To generalize for any multiplication of any size matrix, we check two following conditions [1, Chapter 4]:

- If the thread belong to the valid element of the result matrix $Y$, we add result to that position in $Y$. Otherwise, we do not assign.
- If the thread is transfering the valid element from $W$ or $X-unrolled$ to shared memory, we let it do that. If not, we assign that position in the shared memory as 0.

## Result

Test accuracy and Testing time can be refered to [this table, version D0](#tableResult).

Additionally, this version passed all the designated test cases.

# Reflection

## Final results

- Implement the simplified sequential version of Conv. Layer based on starter project, which is the stardard for further parallelization steps.
- Implement two version of parallelized forward phase of Conv. Layer.
- Post-Seminar modification: implement the independent test cases processes for more reliable correctness verification of forward phase of Conv. layer.

## Unsolved

- Utilise the effectiveness of global and share memory pattern access of CUDA API.
- Utilise the multiple executing streams of CUDA API.
- Parallelize other layers of LeNet-5.

## Difficulties

- The CUDA driver cannot compile the Eigen library: We detach the source code which need the Matrix class and the CUDA source code into seperate files. The CUDA host and kernel execution is as normal functions and put in `cuda_utilities.cu` file, so that other ordinary source files can call wrapped CUDA function, and CUDA does not need to know about Eigen library.
- Because of above problem, the data moving between Matrix class in normal source files and dynamic memory in CUDA source files can be cubersome and may cause bottleneck if we copy raw data from memory to a Matrix class object directly: We implement the CUDA part such that the Matrix class objects can get the result back directly through Map class, without need to do raw copying or transposing. This way-around solution is not straight forward and may cause unaligned or un-coalesced memory accessing, but it is much better than bottle neck of manually copying data between Matrix and dynamic memory.
- Furious bug: CUDA API may make debug process more difficult. We often meet some trivial bugs (such as: uninitialized memory, reset memory, etc) but they are easily hidden below the execution of CUDA threads.
- It is quite not trivial to describe by word the most detail implementation we have done with the source code.  

# References

[1] D. Kirk and Wen-Mei Hwu, Programming massively parallel processors : a hands-on approach. Burlington, Massachusetts: Morgan Kaufmann Elsevier, 2017.

[2] K. Han, “iamhankai/mini-dnn-cpp,” GitHub, Jan. 08, 2024. https://github.com/iamhankai/mini-dnn-cpp (accessed Jan. 13, 2024).
‌